# Example application to 1D Navier equation using Julia
A simple implementation of the Finite Element Method for the resolution of the differential equation

$$
u_{,xx}(x) + f(x) = 0
$$

using a notation similar to Chapter One from the book: The Finite Element Method by Thomas J. R. Hughes.

**under construction** errors and suggestions &rightarrow; [www.fing.edu.uy/~jorgepz](https://www.fing.edu.uy/~jorgepz/)

## Physical interpretation of the equation
Let us consider a truss or unidimensional solid with length $\ell$, submitted to volume load $b(x)$, $u(x)$ is the displacement of each particle, $u_{,x}$ is the axial deformation and $E(x)\times u_{,x}(x)$ is the internal stress at each section located at $x$. The equilibrium equation can be obtained considering a differential of the solid:

$$
\frac{\partial}{\partial x} \left( E(x) u_{,x}(x) \right) + b(x) = 0.
$$

where $b$ is a force per unit of length, such as a unidimensional density.

Let us consider for this example that $E$ is uniform in the domain $[0,\ell]$, and E(x)=E. Then if the equation is divided by $E$ we obtain:

$$
u_{,xx}(x) + f(x) = 0.
$$

where $f(x) = \frac{b(x)}{E}$. Moreover $b(x)$ is a volume force per length volume. Let us consider the self weight: $b(x)=\rho(x) g A(x)$, where $A(x)$ is the area of the cross section.

In [7]:
#using Pkg
#Pkg.add("PyPlot")
using PyPlot
using LinearAlgebra

## Input
Let as consider a steel truss element submitted to gravity load (in vertical position). At $x=0$ the displacement  is fixed thus $u(x)=0$ and in all the domain $b(x) = \rho g A(x)$.

In [8]:
E   = 210e9 ;   #  N/m2
rho = 8000  ;   # kg/m3
g   = 9.81  ;   #  m/s2
l   = 1     ;   #  m
A   = 0.1*0.1 ; # m2

# nodes with known zero displcement
fixdofs = [ 1 ] ; # node corresponding to x=0

nelems = 20 ;  # number of finite elements for the discretization

Auxiliar computations

In [9]:
nnodes = nelems + 1;
lelem  = l / nelems ; # length of each element

totdofs = 1:(nnodes) ;
fredofs = setdiff( totdofs, fixdofs );

Nodes   = lelem * (0:nelems) ;
loadsVector = vec( zeros( nnodes, 1 ) );
Areas   = A * vec( ones(nelems,1) ) ;

## Analysis
System matrix and independent term vector assembly.

In [10]:
using SparseArrays
KG    = spzeros( nnodes, nnodes)
Kbase = [ 1 -1 ; -1 1 ]
FG = loadsVector;

for i in 1:nelems
  elemdofs = [i, i+1]
  A = Areas[i]
  KG[elemdofs, elemdofs ] += E*A/lelem * Kbase ;
  FG[elemdofs]            += rho*g*A*lelem * [ 0.5, 0.5] ;
end


Boundary conditions and system resolution.

In [11]:
Kliblib = KG[fredofs,fredofs ];
Flib    = FG[fredofs];

u = Kliblib \ Flib ;
UG = vec( zeros( nnodes,1));
UG[fredofs] = u;


In [13]:
plot(Nodes,UG, color="black", linewidth=1.5, linestyle="--",marker="o")
xlabel("x")
ylabel("Displacement (m)")
title("Displacements.")


PyCall.PyError: PyError ($(Expr(:escape, :(ccall(#= /home/jor/.julia/packages/PyCall/ttONZ/src/pyfncall.jl:44 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))) <class 'ValueError'>
ValueError("'--x' is not a valid value for ls; supported values are '-', '--', '-.', ':', 'None', ' ', '', 'solid', 'dashed', 'dashdot', 'dotted'")
  File "/usr/lib/python3.7/site-packages/matplotlib/pyplot.py", line 2795, in plot
    is not None else {}), **kwargs)
  File "/usr/lib/python3.7/site-packages/matplotlib/axes/_axes.py", line 1666, in plot
    lines = [*self._get_lines(*args, data=data, **kwargs)]
  File "/usr/lib/python3.7/site-packages/matplotlib/axes/_base.py", line 225, in __call__
    yield from self._plot_args(this, kwargs)
  File "/usr/lib/python3.7/site-packages/matplotlib/axes/_base.py", line 405, in _plot_args
    seg = func(x[:, j % ncx], y[:, j % ncy], kw, kwargs)
  File "/usr/lib/python3.7/site-packages/matplotlib/axes/_base.py", line 312, in _makeline
  